In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11518
2,Huelva,Confirmados PDIA 14 días,616
3,Huelva,Tasa PDIA 14 días,"120,60860354424828"
4,Huelva,Confirmados PDIA 7 días,284
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,14
629,Municipio de Huelva sin especificar,Total Confirmados,144
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11518.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3280.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1408 personas en los últimos 7 días 

Un positivo PCR cada 726 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11518.0,284.0,616.0,510743.0,55.605265,120.608604,80.0
Huelva-Costa,6536.0,184.0,387.0,288115.0,63.863388,134.321365,53.0
Huelva (capital),3280.0,102.0,198.0,143663.0,70.999492,137.822543,28.0
Isla Cristina,492.0,32.0,62.0,21264.0,150.489090,291.572611,14.0
Condado-Campiña,3518.0,58.0,171.0,155057.0,37.405599,110.282025,10.0
Sierra de Huelva-Andévalo Central,1321.0,28.0,38.0,67571.0,41.437895,56.237143,8.0
Palma del Condado (La),566.0,12.0,39.0,10761.0,111.513800,362.419849,5.0
Cerro de Andévalo (El),52.0,11.0,15.0,2364.0,465.313029,634.517766,4.0
Lepe,988.0,12.0,34.0,27431.0,43.746127,123.947359,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),52.0,11.0,15.0,2364.0,465.313029,634.517766,4.0
Cumbres Mayores,19.0,9.0,10.0,1755.0,512.820513,569.800570,2.0
Villablanca,69.0,7.0,14.0,2848.0,245.786517,491.573034,1.0
Palma del Condado (La),566.0,12.0,39.0,10761.0,111.513800,362.419849,5.0
Lucena del Puerto,111.0,5.0,12.0,3371.0,148.323939,355.977455,1.0
Manzanilla,52.0,6.0,7.0,2135.0,281.030445,327.868852,0.0
Isla Cristina,492.0,32.0,62.0,21264.0,150.489090,291.572611,14.0
Palos de la Frontera,268.0,9.0,28.0,11289.0,79.723625,248.029055,1.0
Bonares,68.0,1.0,12.0,6058.0,16.507098,198.085177,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,68.0,1.0,12.0,6058.0,16.507098,198.085177,0.0,0.083333
Almonte,451.0,2.0,12.0,24191.0,8.267538,49.605225,1.0,0.166667
Cartaya,433.0,6.0,34.0,19974.0,30.039051,170.221288,0.0,0.176471
Rociana del Condado,122.0,1.0,5.0,7855.0,12.730745,63.653724,0.0,0.200000
Aljaraque,446.0,3.0,13.0,21260.0,14.111007,61.147695,1.0,0.230769
Bollullos Par del Condado,253.0,1.0,4.0,14272.0,7.006726,28.026906,0.0,0.250000
Palma del Condado (La),566.0,12.0,39.0,10761.0,111.513800,362.419849,5.0,0.307692
Palos de la Frontera,268.0,9.0,28.0,11289.0,79.723625,248.029055,1.0,0.321429
San Juan del Puerto,155.0,1.0,3.0,9300.0,10.752688,32.258065,0.0,0.333333
